In [ ]:
import numpy as np
import json
import time
from data_process import Dataset
from util import accuracy
from tagger import model_training, sentence_tagging

In [ ]:
# load data
with open('hmm_model.json', 'r') as f:
    data = json.load(f)
A = np.array(data['A'])
B = np.array(data['B'])
pi = np.array(data['pi'])
# observation symbols
obs_dict = data['observations']
# state symbols
state_dict = dict()
for idx, item in enumerate(data['states']):
    state_dict[item] = idx
Osequence = np.array(data['Osequence'])
N = len(Osequence)


In [ ]:
print(states_symbols)
reversed_states_symbols = {value : key for (key, value) in states_symbols.items()}
print(reversed_states_symbols)
print(reversed_states_symbols[1])

In [ ]:
key_state_dict = list(states_symbols.keys())
val_state_dict = list(states_symbols.values())
print(key_state_dict)
print(val_state_dict)

In [ ]:
print(obs_dict)
print(Osequence[0])
print(obs_dict[Osequence[0]])
print(B)
print(B[:, obs_dict[Osequence[0]]])

In [ ]:
obs_dict['A'] += 1
print(obs_dict['A'])

In [ ]:
delta = pi * B[:, obs_dict[Osequence[0]]]
print(pi)
print(B[:, obs_dict[Osequence[0]]])
print(delta)

In [ ]:
path = []
S = len(pi)
L = len(Osequence)
delta = np.zeros(shape = (S, L), dtype = 'float')
delta_table = np.zeros(shape = (S, L-1), dtype = 'int')
print(delta_table)
###################################################
# Initialize delta for each state (this is an array with length S)
# self.B[self.obs_dict[Osequence[0]]] =
delta[:,0] = pi * B[:, obs_dict[Osequence[0]]]
print(f'initial delta: ')
print(delta)

# loop through time steps (t)
for t in range(1, L):
    # loop through states
    for i in range(S):
        # find and store max
        # loop through previous states (s')
        local_max = 0
        print(f'------- t, s = ',t+1,",", i," -------")
        for j in range(S):
            print(f'delta used for current max calc: ', delta[j, t-1])
            current_max = A[j, i] * delta[j, t-1]
            
            ######################################
            # test prints
            print(f'current max for t, s, s\' = ',t+1,i,j,": ", current_max)
            ######################################
            
            if current_max > local_max:
                local_max = current_max
                delta_table[i, t-1] = j
                print(f'-> max updated for t, s, s\' = ' ,t+1,i,j,', local max = ',local_max)
        #print(f'max updated for t, s, s\' =' ,t,i,j,'local max = ',local_max)
        delta[i, t] = B[i, obs_dict[Osequence[t]]] * local_max
        print(f'b_s,Ot = ', B[i, obs_dict[Osequence[t]]])
        print(f'local max = ', local_max)
        print(f'delta = ')
        print(delta)
        print(f'delta_table: ')
        print(delta_table)
        
print(f'delta(T) = ', delta)
print(f'# time steps = ', L)
print(f'observed sequence: ', Osequence)
print(f'delta table:') 
print(delta_table)

In [ ]:
path = []
path.append(np.argmax(delta[:,-1]))
print(path)


In [ ]:
for t in range(L-1, 0, -1):
    path.append(delta_table[path[len(path)-1] ,t-1])
    print(path)
path.reverse()
print(path)
print(path[0])

In [ ]:
for i in range(0, len(path)):
    for key, value in state_dict.items():
        if value == path[i]:
            path[i] = key
print(path)

In [ ]:
path = []
###################################################
# Q3.3 Edit here
###################################################

# Initialize
S = len(pi)
L = len(Osequence)
delta = np.zeros([S, L])
delta_path = np.zeros([S, L], dtype=np.int32)
for i in range(S):
    delta[i,0] = pi[i] * B[i, obs_dict[Osequence[0]]]

# recursive DP update
for j in range(1,L):
    for i in range(S):
        s_prime = [A[k,i] * delta[k,j-1] for k in range(S)]
        delta_path[i,j] = np.argmax(s_prime)
        delta[i,j] = B[i, obs_dict[Osequence[j]]] * np.max(s_prime)
        #print(f'local max = ', np.max(s_prime))
        #print(f'delta: ')
        #print(delta)

print(delta_path)
inv_map = {v: k for k, v in state_dict.items()}
max_prob_id = int(np.argmax(delta[:,L-1]))
path = [max_prob_id]
print(path)
for j in range(L-1,0,-1):
    max_prob_id = delta_path[max_prob_id, j]
    path.append(max_prob_id)
    print(path)

path = reversed(path)
path = [inv_map[p] for p in path]

print(path)

In [ ]:
########################## TESTING ########################
import numpy as np
import json
import time
from data_process import Dataset
from util import accuracy
from tagger import model_training, sentence_tagging
# SPEECH TAGGING TESTING
st_time = time.time()
data = Dataset("pos_tags.txt", "pos_sentences.txt", train_test_split=0.8, seed=0)

data.train_data = data.train_data[:100]

data.test_data = data.test_data[:10]


In [ ]:
# TRAIN MODEL
model = model_training(data.train_data, data.tags)
test_data = data.test_data

In [ ]:
tagging = []
S = len(model.pi)
###################################################
# loop through each line in data
for i in range(len(test_data)):
    # loop through each element
    for j in range(test_data[i].length):
        # update obs_dict and emissions if there is an unseen observation
        if test_data[i].words[j] not in model.obs_dict.keys():
            o_indx = len(model.obs_dict)
            model.obs_dict[test_data[i].words[j]] = o_indx 
            new_column = np.full((S, 1), 10**-6)
            model.B = np.append(model.B, new_column, axis = 1)
            print(model.B.shape)

In [ ]:

Osequence = np.asarray(test_data[0].words)
print(Osequence)
print(model.B.shape)
print(model.obs_dict[Osequence[0]])

In [ ]:
state_path = model.viterbi(Osequence)
print(state_path)

In [ ]:
test1 = np.full((10,1), 10**-6)
test2 = np.full((10,2), 1)
print(test1)
print(test2)
test3= np.append(test2, test1, axis = 1)
print(test3)
print(test3.shape)

In [ ]:
tagging = sentence_tagging(data.test_data, model, data.tags)

In [ ]:
### TESTING SENTENCE TAGGING ###

tagging = sentence_tagging(data.test_data, model, data.tags)

total_words = 0
total_correct = 0
for i in range(len(tagging)):
    correct, words, accur = accuracy(tagging[i], data.test_data[i].tags)
    total_words += words
    total_correct += correct
    print("accuracy: ", accur)

print("Your total accuracy: ", total_correct*1.0/total_words)
print("My total accuracy: ", 0.7761904761904762)

en_time = time.time()
print("sentence_tagging total time: ", en_time - st_time)

In [ ]:
obs_dict = {}
obs_indx = 0 
for i in range(L):
    for j in range(train_data[i].length):
        if train_data[i].words[j] not in obs_dict.keys():
            obs_dict[train_data[i].words[j]] = obs_indx 
            obs_indx += 1

B = np.zeros([S, len(obs_dict)])
for i in range(len(train_data)):
    # loop through each element
    for j in range(train_data[i].length):
        s_indx = state_dict[train_data[i].tags[j]]
        o_indx = obs_dict[train_data[i].words[j]]
        B[s_indx, o_indx] += 1
# normalize each row of B to get emissions matrix
sum_rows = B.sum(axis = 1)
B = B / sum_rows[:, np.newaxis]

print(len(obs_dict))
pi = np.zeros(S)
print(len(tags))
print(pi)
print(pi[0])
print(pi[1])
print(B)
print(len(B[:,1]))
print(len(B[1,:]))

In [ ]:
tags = data.tags
train_data = data.train_data
pi = np.zeros(S)
print(len(tags))
print(pi)
print(pi[0])
print(pi[1])
print(pi.size)
print(len(train_data))
pi[0] = 10
print(pi[0])

In [ ]:
L = len(train_data)
S = len(tags)
###################################################
### Calculate pi ###
pi = np.zeros(S, dtype = 'float')
state_counts = {i:0 for i in tags} # dictionary storing counts for each tag (state)
for i in range(len(train_data)):
    first_tag = train_data[i].tags[0] # first tag of each Line object
    state_counts[first_tag] += 1 # increase count in dictionary
    
#print(state_counts)
#print(tags)
# add counts to pi array in same order as tags array
for i in range(len(tags)):
    print(pi)
    pi[i] = state_counts[tags[i]]
    
# normalize by total to get probabilities
pi = pi / len(train_data)
    
print(pi)